# AirBNB Price Scraper

Esta notebook es para ir a airbnb a chekear los precios de los alquileres, porque algunos no eran correctos.

## Librerias

In [ ]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from datetime import datetime
import threading
lock = threading.RLock()
from os import walk

## Leemos datasets originales

In [ ]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print('adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print('adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
dataFrame.loc[:,  'price'] = dataFrame.price.apply(lambda x: x[1:-3].replace(',', '')).astype(int)
print('total rows:', len(dataFrame))
dataFrame.head()

## Elegimos el estado a scrapear de los datasets
modificar el valor de state acorde el estado que quieras scrapear

nota: hicimos esto así para dividir el trabajo y evitar que nos baneen

In [ ]:
state = 'NY'
targets_df = dataFrame[dataFrame.state == state]
targets = targets_df.index

In [ ]:
targets

In [ ]:
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}

def get_price(id):
    url = 'https://www.airbnb.com.ar/api/v2/pdp_listing_booking_details?_format=for_web_dateless&listing_id='+str(id)+'&currency=USD&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&number_of_adults=1'
    response = requests.get(
        url,
        headers=headersChrome
    )
    print()
    data = json.loads(response.text)
    if(data['pdp_listing_booking_details'][0]['p3_display_rate']['currency'] == 'USD'):
        return int(data['pdp_listing_booking_details'][0]['p3_display_rate']['amount'])
    else:
        return None

## Scrapea y guarda cada 20 items scrapeados

In [ ]:
import pickle
real_prices = {}
import time

errors = 0
current = 0

for id in targets:
    current += 1
    if(errors == 25):
        time.sleep(60 * 60)
        errors = 0
    try:
        price = get_price(id)
        real_prices[id] = {
            'price': price,
            'time': time.time()
        }
        if current % 20 == 0:
            pickle.dump( real_prices, open( "../datasets/"+state+"_realPrices.pkl", "wb" ) )
        print('DONE', id)
        errors = 0
    except:
        errors += 1
        real_prices[id] = {
            'price': None,
            'time': time.time()
        }
        print('ERROR', id)
        
pickle.dump( real_prices, open( "../datasets/"+state+"_realPrices.pkl", "wb" ) )